In [1]:
import graphviz
import json
import sqlite3
import sys, os, argparse
import math
import time
import collections

In [2]:
def run_sql_query(db, query):
    conn = sqlite3.connect(db)
    cur = conn.cursor()
    cur.execute(query)
    result_data = json.dumps(cur.fetchall())
    result_json = json.loads(result_data)
    return result_json

In [3]:
def gen_records(graph, nodes, edge_dict_list):
    graph.attr('node', shape='record', fontname='Courier', size='6,6')
    graph.node_attr['fontname'] = "Courier"
    graph.node_attr['color'] = "lightgreen"
    graph.node_attr['fillcolor'] = "lightblue"
    graph.node_attr['style'] = 'filled'
    graph.edge_attr['fontname'] = "Courier"
    graph.graph_attr['rankdir'] = "LR"
    graph.node_attr['fontsize'] = "8"

    for node in nodes:
        graph.node(node.get("id"), 
                   node.get("txt"), 
                   fillcolor=node.get("fillcolor"),
                   rank=node.get("rank"))

    edges = []
    for edge_dict in edge_dict_list:
        graph.edge(
            edge_dict.get("src"),
            edge_dict.get("dest"),
            label=edge_dict.get("label"),
            penwidth=edge_dict.get("penwidth")
        )

def gen_node(node_id, node_txt, fillcolor, rank):
    node = {}
    node["id"] = node_id
    node["txt"] = node_txt
    node["fillcolor"] = fillcolor
    node["rank"] = rank
    return node


In [4]:
def _total_num_of_caps(db):
    count_caps_q = "SELECT COUNT(*) FROM cap_info"
    return run_sql_query(db, count_caps_q)


In [5]:
def show_comparts_no_perms_libraries_only(db, graph):
    get_compart_id_q = "select distinct compart_id, mmap_path from vm where mmap_path != 'unknown' and mmap_path != 'Stack' and mmap_path != 'Guard' and mmap_path not like '%(.plt)' and mmap_path not like '%(.got)'"
    compart_ids = run_sql_query(db, get_compart_id_q) # returns an array of arrays, each with 2 elements: compart_id and mmap_path
    get_caps_q = "SELECT * FROM cap_info"
    caps = run_sql_query(db, get_caps_q)
    
    nodes = []
    edges = []

    for results in compart_ids:
        compart_id = results[0]
        mmap_path = os.path.basename(results[1])

        print(str(compart_id) + ":" + mmap_path)

        if (compart_id < 0):
            fillcolor = "lightgrey"
            rank = "source"
        else:
            fillcolor = "lightblue"
            rank = "max"
            
        nodes.append(gen_node(
                        str(compart_id), 
                        str(compart_id)+": "+mmap_path, 
                        fillcolor,
                        rank))

        lib_start_q = "SELECT start_addr FROM vm WHERE mmap_path LIKE '%" + mmap_path + "%'"
        lib_end_q = "SELECT end_addr FROM vm WHERE mmap_path LIKE '%" + mmap_path + "%'"
        lib_start_addrs = run_sql_query(db, lib_start_q)
        lib_end_addrs = run_sql_query(db, lib_end_q)    

        for cap in caps:
            cap_loc_addr = cap[0]
            cap_path = cap[1]
            cap_addr = cap[2]
            cap_perms = cap[3]
            cap_base = cap[4]
            cap_top = cap[5]
 
            for lib_addr_index in range(len(lib_start_addrs)):
                lib_start_addr = lib_start_addrs[lib_addr_index][0]
                lib_end_addr = lib_end_addrs[lib_addr_index][0]      

                # Also remove the appended (.got/.plt) and compare
                if cap_addr >= lib_start_addr and \
                    cap_addr <= lib_end_addr and \
                    cap_path != mmap_path:

                    find_compart_id_q = "SELECT DISTINCT compart_id FROM vm WHERE mmap_path LIKE '%" + cap_path + "%'"
                    cap_path_compart_id_json = run_sql_query(db, find_compart_id_q)
                    # only need the first compart_id as they should be all the same 
                    cap_compart_id = cap_path_compart_id_json[0][0]
#                   print("cap_path_label: " + str(cap_compart_id) + " single_compart_id: " + str(single_compart_id) + " compart path: " + path)
               
                    for props in edges:
                        if props.get("src") == str(cap_compart_id) and props.get("dest") == str(compart_id):
                            # penwidth_weight = math.log((10**(float(props.get("penwidth"))) + 1), 10)
                            edges.remove(props)
                            break
                    if (str(cap_compart_id) != str(compart_id)):
                        #edges.append({"src":str(cap_compart_id), "dest":str(compart_id), "label":"", "penwidth":1})
                        edges.append({"src":str(cap_compart_id), "dest":str(compart_id), "label":""})
        
    gen_records(graph, nodes, edges)

In [6]:
def show_comparts_no_perms_out_from_plt_only(db, graph):
    get_compart_id_q = "select distinct compart_id, mmap_path from vm WHERE mmap_path != 'unknown' AND mmap_path != 'Stack' AND mmap_path != 'Guard' AND mmap_path NOT LIKE '%(.got)'"
    compart_ids = run_sql_query(db, get_compart_id_q) # returns an array of arrays, each with 2 elements: compart_id and mmap_path
    get_caps_q = "SELECT * FROM cap_info"
    caps = run_sql_query(db, get_caps_q)
    
    nodes = []
    edges = []

    for results in compart_ids:
        compart_id = results[0]
        mmap_path = os.path.basename(results[1])

        print(str(compart_id) + ":" + mmap_path)

        if (compart_id < 0):
            fillcolor = "lightgrey"
            rank = "source"
        elif (compart_id == 2):
            fillcolor = "red"
            rank = "max"
        else:
            fillcolor = "lightblue"
#             rank = "max"
            
        nodes.append(gen_node(
                        str(compart_id), 
                        str(compart_id)+": "+mmap_path, 
                        fillcolor,
                        rank))

        lib_start_q = "SELECT start_addr FROM vm WHERE mmap_path LIKE '%" + mmap_path + "%'"
        lib_end_q = "SELECT end_addr FROM vm WHERE mmap_path LIKE '%" + mmap_path + "%'"
        lib_start_addrs = run_sql_query(db, lib_start_q)
        lib_end_addrs = run_sql_query(db, lib_end_q)    

        for cap in caps:
            cap_loc_addr = cap[0]
            cap_path = cap[1]
            cap_addr = cap[2]
            cap_perms = cap[3]
            cap_base = cap[4]
            cap_top = cap[5]
 
            for lib_addr_index in range(len(lib_start_addrs)):
                lib_start_addr = lib_start_addrs[lib_addr_index][0]
                lib_end_addr = lib_end_addrs[lib_addr_index][0]      

                if cap_addr >= lib_start_addr and \
                    cap_addr <= lib_end_addr and \
                    cap_path != mmap_path:

                    find_compart_id_q = "SELECT DISTINCT compart_id FROM vm WHERE mmap_path LIKE '%" + cap_path + "(.plt)'"
                    cap_path_compart_id_json = run_sql_query(db, find_compart_id_q)
                    if (cap_path_compart_id_json):
                        # only need the first compart_id as they should be all the same 
                        cap_compart_id = cap_path_compart_id_json[0][0]
#                   print("cap_path_label: " + str(cap_compart_id) + " single_compart_id: " + str(single_compart_id) + " compart path: " + path)
            
                        for props in edges:
                            if props.get("src") == str(cap_compart_id) and props.get("dest") == str(compart_id):
                                edges.remove(props)
                                break
                        if (str(cap_compart_id) != str(compart_id)):
                            edges.append({"src":str(cap_compart_id), "dest":str(compart_id), "label":""})
        
    gen_records(graph, nodes, edges)

In [7]:
CONFIG_FILE = 'chericat_cli.config'

def main(argv):
    dbpath="../../chericat_dbs/"
    dbname="konsole.sql"
    if (os.path.isfile(CONFIG_FILE)):
        with open(CONFIG_FILE) as f:
            sys.argv = f.read().split(',')
    else:
        sys.argv = ['chericat_cli', '-d', dbpath+dbname, '-compsfromplt']

    parser = argparse.ArgumentParser(prog='chericat_cli')
    parser.add_argument(
        '-d', 
        help='The database to use for the queries', 
        required=True,
    )
    
    parser.add_argument(
        '-r',
        help='Executes the SQL query on the provided db',
        nargs=1,
    )
    
    parser.add_argument(
        '-compsfromplt',
        help="Show compartments graph with capabilities from PLT in each library of the target process",
        action='store_true',
    )

    parser.add_argument(
        '-compslibs',
        help="Show compartments graph with capabilities from libraries (no stack or GOT as separate nodes)",
        action='store_true',
    )

    args = parser.parse_args()

    if args.d:
        db = args.d

    if args.r:
        print(run_sql_query(db, args.r[0]))

    if args.compsfromplt:
        start = time.perf_counter()
        digraph = graphviz.Digraph('G', filename=dbname+'.comp_graph.gv')
        show_comparts_no_perms_out_from_plt_only(db, digraph)
        end = time.perf_counter()
        print("Comparts graph generation time taken: " + str(end-start) + "s")
        digraph.render(directory='graph-output', view=True)

    if args.compslibs:
        start = time.perf_counter()
        digraph = graphviz.Digraph('G', filename=dbname+'.comp_no_perms_paths_only_graph.gv')
        show_comparts_no_perms_libraries_only(db, digraph)
        end = time.perf_counter()
        print("Capability comparts graph generation time taken: " + str(end-start) + "s")
        digraph.render(directory='graph-output', view=True)
    
    return None

if __name__ == '__main__':
    main(sys.argv)


2:konsole
2:konsole(.plt)
-2:ld-elf.so.1
-1:libkonsoleapp.so.23.04.3
-1:libkonsoleapp.so.23.04.3(.plt)
4:libKF5NotifyConfig.so.5.108.0
4:libKF5NotifyConfig.so.5.108.0(.plt)
-1:libkonsoleprivate.so.23.04.3
-1:libkonsoleprivate.so.23.04.3(.plt)
6:libKF5NewStuffWidgets.so.5.108.0
6:libKF5NewStuffWidgets.so.5.108.0(.plt)
7:libKF5Notifications.so.5.108.0
7:libKF5Notifications.so.5.108.0(.plt)
8:libKF5TextWidgets.so.5.108.0
8:libKF5TextWidgets.so.5.108.0(.plt)
9:libKF5SonnetUi.so.5.108.0
9:libKF5SonnetUi.so.5.108.0(.plt)
10:libKF5NewStuff.so.5.108.0
10:libKF5NewStuff.so.5.108.0(.plt)
11:libKF5NewStuffCore.so.5.108.0
11:libKF5NewStuffCore.so.5.108.0(.plt)
12:libKF5Attica.so.5.108.0
12:libKF5Attica.so.5.108.0(.plt)
13:libKF5Pty.so.5.108.0
13:libKF5Pty.so.5.108.0(.plt)
14:libutil.so.9
14:libutil.so.9(.plt)
15:libz.so.6
15:libz.so.6(.plt)
16:libicuuc.so.73.2
16:libicuuc.so.73.2(.plt)
17:libicui18n.so.73.2
17:libicui18n.so.73.2(.plt)
18:libKF5Bookmarks.so.5.108.0
18:libKF5Bookmarks.so.5.108.0(.pl

KeyboardInterrupt: 